In [1]:
# sentiment_pipeline.py

import torch
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 1. Load the IMDb dataset
dataset = load_dataset("imdb")

# 2. Load tokenizer and model
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)

# 3. Tokenization function
def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding=True)

# 4. Preprocess dataset
encoded_dataset = dataset.map(tokenize, batched=True)
encoded_dataset = encoded_dataset.remove_columns(["text"])
encoded_dataset.set_format("torch")

# 5. Evaluation metrics
def compute_metrics(pred):
    logits, labels = pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions)
    return {"accuracy": acc, "f1": f1}


2025-06-27 22:44:01.867689: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751064242.049013      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751064242.105081      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using device: cuda


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding, logging

# Optional: enable informative logging for progress display
logging.set_verbosity_info()

# 6. TrainingArguments (with logging and tqdm settings)
training_args = TrainingArguments(
    output_dir="./results",
    run_name="sentiment_analysis_run",     # avoids wandb warning
    eval_strategy="epoch",                 # for evaluation each epoch
    save_strategy="epoch",                 # save checkpoint each epoch
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir="./logs",
    logging_steps=10,                      # log every 10 steps
    logging_first_step=True,               # also log at step 1
    report_to="none",                      # disable wandb/tensorboard
    disable_tqdm=False,                    # enable progress bar
)

# 7. Trainer setup (no tokenizer= to avoid deprecation warning)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"].shuffle(seed=42).select(range(2000)),
    eval_dataset=encoded_dataset["test"].select(range(500)),
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# 8. Train the model
trainer.train()

# 9. Save model
model.save_pretrained("sentiment_model")
tokenizer.save_pretrained("sentiment_model")

# 10. Load and test on sample
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    prediction = torch.argmax(logits, dim=-1).item()
    return "positive" if prediction == 1 else "negative"

# Sample prediction
print("Sample prediction:", predict("The movie was absolutely amazing!"))


PyTorch: setting up devices
***** Running training *****
  Num examples = 2,000
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 500
  Number of trainable parameters = 109,483,778


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.534100,0.334949,0.900000,0.000000
2,0.257700,0.387983,0.906000,0.000000



***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-250
Configuration saved in ./results/checkpoint-250/config.json
Model weights saved in ./results/checkpoint-250/model.safetensors
Saving Trainer.data_collator.tokenizer by default as Trainer.processing_class is `None`
tokenizer config file saved in ./results/checkpoint-250/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-250/special_tokens_map.json

***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/model.safetensors
Saving Trainer.data_collator.tokenizer by default as Trainer.processing_class is `None`
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json


Training compl

Sample prediction: positive
